### Python Code for the paper:
##### Di Zhu, Zhou Huang, Li Shi, Lun Wu & Yu Liu (2018) Inferring spatial interaction patterns from sequential snapshots of spatial distributions, International Journal of Geographical Information Science, 32:4, 783-805, DOI: 10.1080/13658816.2017.1413192
https://doi.org/10.1080/13658816.2017.1413192

##### ref: https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html

In [2]:
from pulp import *

### 1. simple scenario example
<img src="./IIDS_pic.jpg" width = "40%" />

In [33]:
# declare problem
prob = LpProblem("IIDS", LpMinimize)
# declaration variables
x1 = LpVariable("x1", 0, None, LpInteger)
x2 = LpVariable("x2", 0, None, LpInteger)
x3 = LpVariable("x3", 0, None, LpInteger)
x4 = LpVariable("x4", 0, None, LpInteger)
x5 = LpVariable("x5", 0, None, LpInteger)
x6 = LpVariable("x6", 0, None, LpInteger)
# objective func
prob += x1 + x2 + x3 + x4 + x5 + x6, "Total Cost of movements"
# subjective func
prob += -x1 + x2 - x5 + x6 == -3, "city1"
prob += x1 - x2 - x3 + x4 == 4, "city2"
prob += x3 - x4 + x5 - x6 == -1, "city3"
# solve the LP
prob.solve()
print("Status:", LpStatus[prob.status])
# print the results
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost of movements: ", value(prob.objective))

Status: Optimal
x1 = 3.0
x2 = 0.0
x3 = 0.0
x4 = 1.0
x5 = 0.0
x6 = 0.0
Total Cost of movements:  4.0


### 2. real scenario code
<img src="./IIDS_pic2.png" width = "75%" />

In [5]:
import numpy as np
import pandas as pd

In [19]:
#  A for AfterNewYearsEve
#  B for BeforeSpringFestival
snapshot_data=pd.read_csv('./data/city_check_before_after.csv',encoding='GBK')
before_data=pd.read_csv('./data/before.txt',encoding='GBK')
after_data=pd.read_csv('./data/after.txt',encoding='GBK')

In [30]:
geocoord=pd.read_csv('./data/xycord_old.csv',encoding='GBK',header=None)
geocoord.columns=["id","name",'x','y']

In [40]:
after_data.iloc[0,0]

15802435

In [32]:
after_data,before_data,geocoord

(          361
 0    15802435
 1     1440478
 2      907863
 3     1838387
 4     2679616
 5      852361
 6      318175
 7     2474892
 8     5382475
 9     4064227
 10    1388721
 11    4609278
 12    1721895
 13    1915371
 14    2903271
 15     988425
 16     201588
 17    1461660
 18     566650
 19    1378611
 20    2541627
 21    2738598
 22    2492338
 23    1443062
 24    4178475
 25    3260992
 26    1599166
 27     902307
 28     268751
 29   19336025
 ..        ...
 331   1788204
 332   2467629
 333   4407217
 334   2262390
 335   3470317
 336   3353895
 337   6724948
 338  10637223
 339    828139
 340     36332
 341   1003748
 342    225773
 343    215673
 344    474573
 345     75039
 346   2257225
 347     81294
 348    863676
 349   1052296
 350   2270037
 351   1436096
 352   9861928
 353    247208
 354   2832118
 355   2447399
 356   4453675
 357   1543348
 358   1014832
 359   1480606
 360   4231526
 
 [361 rows x 1 columns],           361
 0    28004719
 1     1357287

In [35]:
# city_ids
cities=[]
for i in range(0,snapshot_data.shape[0]):
    cities.append(str(i))

In [76]:
# the variation for each city "key ~ value": "city_id ~ change_num"
variations={}
for i in cities:
    change=after_data.iloc[int(i),0]-before_data.iloc[int(i),0]
    variations[i]=change
variations

{'0': -12202284,
 '1': 83191,
 '10': -34822,
 '100': 1229695,
 '101': 831480,
 '102': 621315,
 '103': -2026683,
 '104': 959053,
 '105': 304198,
 '106': 497741,
 '107': 447152,
 '108': 2114370,
 '109': 709202,
 '11': 842093,
 '110': 217627,
 '111': -12986026,
 '112': -4199941,
 '113': 1123119,
 '114': -8291734,
 '115': -17238577,
 '116': -2011950,
 '117': 2388056,
 '118': 2264922,
 '119': 427241,
 '12': 316662,
 '120': 1115724,
 '121': -533880,
 '122': 1298924,
 '123': -21013367,
 '124': 1046461,
 '125': 1968660,
 '126': 530195,
 '127': -1465942,
 '128': 756516,
 '129': 3249525,
 '13': 51881,
 '130': 762337,
 '131': 965054,
 '132': -50054,
 '133': -1491477,
 '134': 650330,
 '135': 1165543,
 '136': -396617,
 '137': 917205,
 '138': 1630682,
 '139': 1058117,
 '14': 97809,
 '140': 3757221,
 '141': 476193,
 '142': 3337556,
 '143': 1051848,
 '144': 1651542,
 '145': -46803,
 '146': -723772,
 '147': -95099,
 '148': 190034,
 '149': -11118,
 '15': -184882,
 '150': -22043,
 '151': 75338,
 '152': -

In [109]:
# symmetric cost matrix generated using './data/generateCostMatrix.py'
# use \beta=0.8 gravity model $c\approx \frac{d^\beta}{A_iA_j}$ as an example
cost_matrix=np.loadtxt('./data/Cost_Matrix0.8.txt',delimiter=' ',skiprows=1)
cost_matrix=cost_matrix.tolist()

In [104]:
#declare problem
prob = LpProblem("IIDS_ChineseSF2016", LpMinimize)

# Creates a list of tuples containing all the possible flows for movements
flows = [(o,d) for o in cities for d in cities]

# A dictionary called flow_vars is created to contain the referenced variables (the flows)
flow_vars = LpVariable.dicts("flow",(cities,cities),0,None,LpInteger) # n*n
#call flow_vars['2']['1'] # flow_2_1 means flow from city 2 to city 1

# The objective func
prob += lpSum([flow_vars[o][d]*cost_matrix[int(o)][int(d)] for (o,d) in flows]), "Sum of movement costs"

# The subjective func for each city
# The constraints are added to make it consist for all cities
# since after data is not exactly the same as before data due to float/int convertion, the constraints can be >= or <= depending on the data
for c in cities:
    prob += lpSum([flow_vars[x][c] for x in cities]) - lpSum([flow_vars[c][x] for x in cities]) >= variations[c], "Sum of in/outflow of city %s"%c

In [105]:
# solve the LP
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [108]:
# print the results
for v in prob.variables():
    if v.varValue!=0.0:
        print(v.name, "=", v.varValue)
print("Total Cost of movements: ", value(prob.objective))

flow_0_165 = 623943.0
flow_0_166 = 1692816.0
flow_0_168 = 5477792.0
flow_0_171 = 810596.0
flow_0_173 = 1937648.0
flow_0_175 = 1333987.0
flow_0_202 = 5325982.0
flow_0_203 = 3387484.0
flow_0_213 = 2276211.0
flow_0_252 = 633668.0
flow_0_29 = 1697185.0
flow_0_66 = 2131936.0
flow_0_69 = 2496120.0
flow_0_73 = 443580.0
flow_0_78 = 960609.0
flow_0_79 = 30089.0
flow_0_80 = 2738427.0
flow_0_84 = 2245101.0
flow_0_91 = 2128430.0
flow_0_98 = 3103815.0
flow_103_101 = 831480.0
flow_103_105 = 304198.0
flow_103_194 = 891005.0
flow_10_329 = 34822.0
flow_111_115 = 12986026.0
flow_112_123 = 4199941.0
flow_114_115 = 8291734.0
flow_115_113 = 1123119.0
flow_115_122 = 1298924.0
flow_115_124 = 1046461.0
flow_115_128 = 756516.0
flow_115_129 = 3249525.0
flow_115_130 = 762337.0
flow_115_131 = 965054.0
flow_115_135 = 1165543.0
flow_115_137 = 917205.0
flow_115_138 = 1630682.0
flow_115_140 = 3757221.0
flow_115_142 = 3337556.0
flow_115_143 = 1051848.0
flow_115_144 = 713538.0
flow_115_180 = 1714285.0
flow_115_182 = 13

<img src="./IIDS_result_example.png" width = "60%" />